# Load Data

In [1]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xvf  'dakshina_dataset_v1.0.tar'

--2021-05-14 06:03:27--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.211.128, 173.194.213.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   129MB/s    in 17s     

2021-05-14 06:03:44 (113 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.tx

# New Section

In [2]:
%pip install wandb -q
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8MB 6.8MB/s 
     |████████████████████████████████| 174kB 27.2MB/s 
     |████████████████████████████████| 133kB 20.9MB/s 
     |████████████████████████████████| 102kB 7.8MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 


In [16]:
wandb -version

NameError: ignored

In [3]:
input_token_index = None
target_token_index = None
MAX_LEN_input = None
MAX_LEN_target = None
num_encoder_tokens = 70
num_decoder_tokens = 30
input_tokenizer = None

In [4]:
def tokenize(data,vocab_size):
  tokenizer = Tokenizer(num_words=vocab_size,char_level=True)
  tokenizer.fit_on_texts(data)
  temp=tokenizer.texts_to_sequences(data)
  # print(data[:3])
  # print(temp[:3])
  dictionary = tokenizer.word_index
  return temp , dictionary , tokenizer

In [5]:
def load_and_preprocess():
  global input_token_index , target_token_index , MAX_LEN_input , MAX_LEN_target ,num_decoder_tokens,num_encoder_tokens , input_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.train.tsv'
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split('\t')
    input_text, target_text = temp[0],temp[1] 
    target_text = "\t" + target_text + "\n"
    input_text = input_text+"\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
  
  MAX_LEN_input = max([len(txt) for txt in input_texts])
  MAX_LEN_target = max([len(txt) for txt in target_texts])

  # toeknize
  encoder_input , input_token_index , input_tokenizer = tokenize(input_texts , num_encoder_tokens)
  decoder_input , target_token_index,_ = tokenize(target_texts , num_decoder_tokens) 

  # padding
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  decoder_input_data = pad_sequences(decoder_input, maxlen=MAX_LEN_target, dtype='int32', padding='post', truncating='post',value=target_token_index["\n"])

  decoder_target_data = np.zeros((len(input_texts), MAX_LEN_target, num_decoder_tokens), dtype="float32")
  for i,  target_text in enumerate(target_texts):
    for t, char in enumerate(target_text):
      if t > 0:
        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return encoder_input_data , decoder_input_data, decoder_target_data



In [6]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer, Concatenate
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


In [8]:
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Input,GRU,SimpleRNN,Dropout,Embedding

def create_model_attention(m_name="LSTM",n_e_layers=1,n_d_layers=1,latent_dim = 100,embedding_size = 16,dropout = 0 , recurrent_dropout = 0):
  global num_decoder_tokens,num_decoder_tokens
  keras.backend.clear_session()
  
  # Define an input sequence and process it.
  input1 = Input(shape=(None,),name= "input_1")
  encoder_inputs = Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(input1)

  encoder = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True,return_sequences=True)
  e_o = encoder(encoder_inputs)
  prev = e_o
  for i in range(1,n_e_layers):
    e = globals()[m_name](latent_dim, dropout=dropout,recurrent_dropout = recurrent_dropout,return_state=True,return_sequences=True)
    e_o = e(prev[0])
    prev = e_o
  
  input2 = Input(shape=(None,),name="input_2")
  decoder_inputs = Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(input2)
  d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_sequences=True, return_state=True)(decoder_inputs,initial_state = e_o[1:])
  p_d = d_l[0]
  for i in range(1,n_d_layers):
    d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True, return_sequences=True)(p_d,initial_state = e_o[1:])
    p_d = d_l[0]

  attn_layer = AttentionLayer(name="attention_layer")
  attn_op, attn_state = attn_layer([e_o[0], d_l[0]])
  decoder_concat_input = Concatenate(axis=-1)([d_l[0], attn_op])


  dec_dense = Dense(num_decoder_tokens, activation='softmax')
  final_output = dec_dense(decoder_concat_input)
 

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([input1,input2], final_output)

  return model


In [14]:
def train_attention():
  run = wandb.init()
  c = run.config
  name = "model_"+c.model+"_hs_"+str(c.hidden_size)+"_em_"+str(c.embedding_size)+"_d_"+str(c.dropout)
  run.name = name
  print(name)
  batch_size = 64
  epochs = 10

  # used single encoder and decoder layer
  encoder_layers , decoder_layers = 1 , 1


  encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

  model = create_model_attention(c.model,encoder_layers,decoder_layers,c.hidden_size,c.embedding_size,c.dropout,0)
  model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[WandbCallback(),es]
  )
  return

In [10]:
sweep_config_attention={
    'method' : 'random' ,
    'metric' : { 'name' : 'val_accuracy' , 'goal' : 'maximize' } ,
    'parameters' : {
        'model' : { 'values' : ['LSTM','GRU','SimpleRNN'] },
        'dropout' : { 'values' : [0.1,0.2]},
        'embedding_size' : {'values' : [16,32,64]},
        'hidden_size' : {'values' : [64,128,256]}
    }
}


In [13]:

sweepid = wandb.sweep(sweep_config_attention,project="DL_A3_Q5",entity ="sonagara")
wandb.agent(sweepid,train_attention)

Create sweep with ID: bdaoidg4
Sweep URL: https://wandb.ai/sonagara/DL_testing/sweeps/bdaoidg4


wandb: Agent Starting Run: 50r33sxb with config:
wandb: 	dropout: 0.2
wandb: 	embedding_size: 16
wandb: 	hidden_size: 128
wandb: 	model: LSTM


model_LSTM_hs_128_em_16_d_0.2
1316/1316 [==============================] - 316s 236ms/step - loss: 0.9945 - accuracy: 0.7298 - val_loss: 0.9995 - val_accuracy: 0.7360


epoch,0
loss,0.85029
accuracy,0.75258
val_loss,0.9995
val_accuracy,0.73604
_runtime,325
_timestamp,1620972700
_step,0
best_val_loss,0.9995
best_epoch,0


epoch,▁
loss,▁
accuracy,▁
val_loss,▁
val_accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: e051bkce with config:
wandb: 	dropout: 0.2
wandb: 	embedding_size: 32
wandb: 	hidden_size: 128
wandb: 	model: LSTM


model_LSTM_hs_128_em_32_d_0.2
 353/1316 [=======>......................] - ETA: 3:36 - loss: 1.1977 - accuracy: 0.6997

wandb: Ctrl + C detected. Stopping sweep.
